In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

mpl.rc('font', family='serif', size=15)
mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{bm}')

out_dir = '../assets/output/'

In [ ]:
def newton(g, J, u0, ue=None, eps=1e-5, max_iter=100, print_flag=False):
    u, gu = u0.copy(), g(u0)
    df = pd.DataFrame(columns=[f'||u-ue||2'] + [f'||g(u)||∞'] + [f'||Δu||∞'])
    if print_flag:
        if ue is None: ue = np.zeros(u0.shape)
        df.loc[0] = [la.norm(u0 - ue, np.inf), la.norm(gu, np.inf), np.nan]
    for k in range(max_iter):
        Ju = J(u)
        du = la.solve(Ju, -gu)
        u += du
        gu = g(u)
        if print_flag:
            df.loc[k + 1] = [la.norm(u - ue, np.inf), la.norm(gu, np.inf), la.norm(du, np.inf)]
        if la.norm(du, np.inf) < eps:
            return (u, df) if print_flag else (u, k, la.norm(u - ue, np.inf))
    raise RuntimeError('Failed to converge.')


def g(u):
    g1 = np.empty((n - 1, 1))
    g1[0] = 2 * u[0] - u[1] + h**2 * u[0]**3 - h**2 * f[0]
    g1[n - 2] = -u[n - 3] + 2 * u[n - 2] + h**2 * u[n - 2]**3 - h**2 * f[n - 2]
    for i in range(1, n - 2):
        g1[i] = -u[i - 1] + 2 * u[i] - u[i + 1] + h**2 * u[i]**3 - h**2 * f[i]
    return g1


def J(u):
    J1 = J0.copy()
    for i in range(0, n - 1):
        J1[i, i] += 3 * h**2 * u[i]**2
    return J1


uk_dict, df_dict = {}, {}

for n in [10, 20, 40, 80, 160]:
    h = 1 / n
    x = np.linspace([h], [1 - h], n - 1)

    f = np.sin(np.pi * x)
    f = np.pi**2 * f + f**3

    u0 = np.zeros((n - 1, 1))
    ue = np.sin(np.pi * x)

    J0 = np.zeros((n - 1, n - 1))
    J0[0, 0] = 2
    for i in range(1, n - 1):
        J0[i - 1, i] = -1
        J0[i, i - 1] = -1
        J0[i, i] = 2

    uk_dict[n], df_dict[n] = newton(g, J, u0, ue, eps=1e-8, print_flag=True)

uk_dict, df_dict

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)

n = 10
h, x = 1 / n, np.linspace(0, 1, n + 1)
ax[0].plot(x, np.vstack((0, uk_dict[n], 0)), 'C1o-', mec='1.0', linewidth=1, label=r'$u_{' + f'{h}' + '}(x)$')

n = 160
h, x = 1 / n, np.linspace(0, 1, n + 1)
ax[1].plot(x, np.vstack((0, uk_dict[n], 0)), linewidth=1, label=r'$u_{' + f'{h}' + '(x)}$')

for i in [0, 1]:
    ax[i].set_xlabel(r'$x$')
    ax[i].set_xlim(0.0, 1.0)
    ax[i].set_ylim(0.0, 1.05)
    ax[i].grid()
    ax[i].legend()

fig.savefig(out_dir + 'newton.pdf')
plt.show()

In [ ]:
dict = {'x': np.linspace(0.1, 0.9, 9)}
for n in [10, 20, 40, 80, 160]:
    h = 1 / n
    dict[f'u{h}'] = uk_dict[n][:, 0][int(n / 10 - 1)::int(n / 10)]
dict['ue'] = np.sin(np.pi * dict['x'])
df = pd.DataFrame(dict)
df.to_csv(out_dir + 'newton.csv')
df

In [ ]:
df = pd.DataFrame([[n, 1 / n, df_dict[n]['||u-ue||2'].iloc[-1]] for n in [10, 20, 40, 80, 160]], columns=['n', 'h', 'eh'])
df['logh'] = np.log(df['h'])
df['logeh'] = np.log(df['eh'])
df.to_csv(out_dir + 'newton-error.csv')
df

In [ ]:
def fit(N, h, e):
    a, b, c, d = 0, 0, 0, 0
    for i in range(N):
        x, y = np.log(h[i]), np.log(e[i])
        a += x
        b += y
        c += x**2
        d += x * y
    return ((N * d - a * b), (b * c - a * d)) / (N * c - a**2)


c = fit(df.shape[0], df['h'].array, df['eh'].array)
c

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), constrained_layout=True)
ax.set_xlabel(r'$\log h$')
ax.scatter(df['logh'], df['logeh'], zorder=2, label=r'$\log e_h$', marker='o')
ax.plot([-5.2, -2.2], np.polyval(c, [-5.2, -2.2]), label=r'$\beta_0 + \beta \log h$', zorder=1, linewidth=1, c='C1')
ax.grid()
ax.legend()

fig.savefig(out_dir + 'newton-error.pdf')
plt.show()

In [ ]:
def jacobi(f, u0, ue=None, eps=1e-5, max_iter=100000, print_flag=False):
    u = np.vstack((0, u0, 0))
    v = np.empty(u0.shape)
    df = pd.DataFrame(columns=[f'||u-ue||2'] + [f'||Δu||∞'])
    if print_flag:
        if ue is None: ue = np.zeros(u0.shape)
        df.loc[0] = [la.norm(u0 - ue, np.inf), np.nan]
    for k in range(max_iter):
        m = 0
        for i in range(1, u.shape[0] - 1):
            v[i - 1] = (u[i - 1] + u[i + 1] + f[i - 1]) / 2
            m = np.max([m, np.abs(v[i - 1, 0] - u[i, 0])])
        if print_flag: df.loc[k + 1] = [la.norm(v - ue, np.inf), m]
        for i in range(1, u.shape[0] - 1):
            u[i] = v[i - 1]
        if m < eps: return (v, df) if print_flag else (v, k, la.norm(v - ue, np.inf))
    raise RuntimeError('Failed to converge.')


def gauss_seidel(f, u0, ue=None, eps=1e-5, max_iter=100000, print_flag=False):
    u = np.vstack((0, u0, 0))
    v = np.empty(u0.shape)
    df = pd.DataFrame(columns=[f'||u-ue||2'] + [f'||Δu||∞'])
    if print_flag:
        if ue is None: ue = np.zeros(u0.shape)
        df.loc[0] = [la.norm(u0 - ue, np.inf), np.nan]
    for k in range(max_iter):
        m = 0
        v[0] = (u[2] + f[0]) / 2
        for i in range(2, u.shape[0] - 1):
            v[i - 1] = (v[i - 2] + u[i + 1] + f[i - 1]) / 2
            m = np.max([m, np.abs(v[i - 1, 0] - u[i, 0])])
        if print_flag: df.loc[k + 1] = [la.norm(v - ue, np.inf), m]
        for i in range(1, u.shape[0] - 1):
            u[i] = v[i - 1]
        if m < eps: return (v, df) if print_flag else (v, k, la.norm(v - ue, np.inf))
    raise RuntimeError('Failed to converge.')


uk_dict1, uk_dict2 = {}, {}

for n in [10, 20, 40, 80, 160]:
    h = 1 / n
    x = np.linspace([h], [1 - h], n - 1)

    f = np.pi**2 * np.sin(np.pi * x)
    f *= h**2

    u0 = np.zeros((n - 1, 1))
    ue = np.sin(np.pi * x)

    uk_dict1[n] = jacobi(f, u0, ue, eps=1e-10)
    uk_dict2[n] = gauss_seidel(f, u0, ue, eps=1e-10)

uk_dict1, uk_dict2

In [ ]:
dict = {'x': np.linspace(0.1, 0.9, 9)}
for n in [10, 20, 40, 80, 160]:
    h = 1 / n
    dict[f'u{h}'] = uk_dict1[n][0][:, 0][int(n / 10 - 1)::int(n / 10)]
dict['ue'] = np.sin(np.pi * dict['x'])
df = pd.DataFrame(dict)
df.to_csv(out_dir + 'jacobi.csv')
df

In [ ]:
dict = {'x': np.linspace(0.1, 0.9, 9)}
for n in [10, 20, 40, 80, 160]:
    h = 1 / n
    dict[f'u{h}'] = uk_dict2[n][0][:, 0][int(n / 10 - 1)::int(n / 10)]
dict['ue'] = np.sin(np.pi * dict['x'])
df = pd.DataFrame(dict)
df.to_csv(out_dir + 'gauss-seidel.csv')
df

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)

n = 10
h, x = 1 / n, np.linspace(0, 1, n + 1)
ax[0].plot(x, np.vstack((0, uk_dict1[n][0], 0)), 'C1o-', mec='1.0', linewidth=1, label=r'$u_{' + f'{h}' + '}(x)$')

n = 160
h, x = 1 / n, np.linspace(0, 1, n + 1)
ax[1].plot(x, np.vstack((0, uk_dict1[n][0], 0)), linewidth=1, label=r'$u_{' + f'{h}' + '(x)}$')

for i in [0, 1]:
    ax[i].set_xlabel(r'$x$')
    ax[i].set_xlim(0.0, 1.0)
    ax[i].set_ylim(0.0, 1.05)
    ax[i].grid()
    ax[i].legend()

fig.savefig(out_dir + 'jacobi.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)

n = 10
h, x = 1 / n, np.linspace(0, 1, n + 1)
ax[0].plot(x, np.vstack((0, uk_dict2[n][0], 0)), 'C1o-', mec='1.0', linewidth=1, label=r'$u_{' + f'{h}' + '}(x)$')

n = 160
h, x = 1 / n, np.linspace(0, 1, n + 1)
ax[1].plot(x, np.vstack((0, uk_dict2[n][0], 0)), linewidth=1, label=r'$u_{' + f'{h}' + '(x)}$')

for i in [0, 1]:
    ax[i].set_xlabel(r'$x$')
    ax[i].set_xlim(0.0, 1.0)
    ax[i].set_ylim(0.0, 1.05)
    ax[i].grid()
    ax[i].legend()

fig.savefig(out_dir + 'gauss-seidel.pdf')
plt.show()

In [ ]:
df = pd.DataFrame([[n, 1 / n, uk_dict1[n][2], uk_dict2[n][2]] for n in [10, 20, 40, 80, 160]], columns=['n', 'h', 'eh1', 'eh2'])
df['logh'] = np.log(df['h'])
df['logeh1'] = np.log(df['eh1'])
df['logeh2'] = np.log(df['eh2'])
df.to_csv(out_dir + 'linear-itr-error.csv')
df

In [ ]:
def fit(N, h, e):
    a, b, c, d = 0, 0, 0, 0
    for i in range(N):
        x, y = np.log(h[i]), np.log(e[i])
        a += x
        b += y
        c += x**2
        d += x * y
    return ((N * d - a * b), (b * c - a * d)) / (N * c - a**2)


c1 = fit(df.shape[0], df['h'].array, df['eh1'].array)
c2 = fit(df.shape[0], df['h'].array, df['eh2'].array)
c1, c2

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 8), constrained_layout=True)

ax[0].set_xlabel(r'$\log h$')
ax[0].scatter(df['logh'], df['logeh1'], zorder=2, label=r'$\log e_h$ (Jacobi Iter.)', marker='o')
ax[0].plot([-5.2, -2.2], np.polyval(c1, [-5.2, -2.2]), label=r'$\beta_0 + \beta \log h$', zorder=1, linewidth=1, c='C1')
ax[0].grid()
ax[0].legend()

ax[1].set_xlabel(r'$\log h$')
ax[1].scatter(df['logh'], df['logeh2'], zorder=2, label=r'$\log e_h$ (Gauss-Seidel Iter.)', marker='o', c='C2')
ax[1].plot([-5.2, -2.2], np.polyval(c2, [-5.2, -2.2]), label=r'$\beta_0 + \beta \log h$', zorder=1, linewidth=1, c='C3')
ax[1].grid()
ax[1].legend()

fig.savefig(out_dir + 'linear-itr-error.pdf')
plt.show()

In [ ]:
df = pd.DataFrame([[n, uk_dict1[n][1], uk_dict2[n][1]] for n in [10, 20, 40, 80, 160]], columns=['n', 'k1', 'k2'])
df.to_csv(out_dir + 'linear-itr-k.csv')
df

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), constrained_layout=True)

ax.set_xticks(df['n'])
ax.set_xlabel(r'$n$')
ax.set_ylim(0, 90000)
ax.plot(df['n'], df['k1'], 'C0o-', mec='1.0', linewidth=1, label=r'$k$ of Jacobi Iter.')
ax.plot(df['n'], df['k2'], 'C1o-', mec='1.0', linewidth=1, label=r'$k$ of Gauss-Seidel Iter.')
ax.grid()
ax.legend()

fig.savefig(out_dir + 'linear-itr-k.pdf')
plt.show()